## Setup: Finnhub API Key

This notebook uses real market data from [Finnhub](https://finnhub.io/) (free tier: 60 API calls/min).

**To use real data:**
1. Get a free API key at https://finnhub.io/register
2. Set environment variable: `export FINNHUB_API_KEY=your_key_here`
3. Or enter the key when prompted

**Without a key:** The notebook falls back to synthetic data generation.

# Pair Trading — Example Notebook

Classic statistical pair trading using z-score on spread. The notebook has clear equations, an implementable strategy, a backtest with PnL, Sharpe and drawdown, plus visualisations.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import math
import random

try:
    from python.backtest.core import Backtest, sharpe_ratio, max_drawdown

# Import Finnhub helper for real market data
try:
    from python.finnhub_helper import fetch_historical_simulation, get_finnhub_api_key, create_orderbook_from_quote
    FINNHUB_AVAILABLE = True
except Exception as e:
    print(f"Finnhub helper not available: {e}")
    FINNHUB_AVAILABLE = False

except Exception:
    # fallback (same minimal as above)
    class Backtest:
        def __init__(self, initial_cash=100000.0):
            self.cash = initial_cash
            self.positions = {}
            self.trades = []
            self.history = []
        def execute_trade(self, timestamp, symbol, qty, price, side):
            cost = qty * price
            if side.lower() == 'buy':
                self.cash -= cost
                prev = self.positions.get(symbol, (0.0, 0.0))
                new_qty = prev[0] + qty
                new_avg = (prev[0]*prev[1] + qty*price) / (new_qty if new_qty!=0 else 1)
                self.positions[symbol] = (new_qty, new_avg)
            else:
                self.cash += cost
                prev = self.positions.get(symbol, (0.0, 0.0))
                new_qty = prev[0] - qty
                if new_qty <= 0:
                    self.positions.pop(symbol, None)
                else:
                    self.positions[symbol] = (new_qty, prev[1])
            self.trades.append({"ts": timestamp, "symbol": symbol, "qty": qty, "price": price, "side": side})
            self.mark_to_market(timestamp, {})
        def mark_to_market(self, timestamp, market_prices: dict):
            equity = self.cash
            for s,(qty,avg) in self.positions.items():
                price = market_prices.get(s, avg)
                equity += qty*price
            self.history.append((timestamp, equity))
            return equity
        def results_df(self):
            df = pd.DataFrame(self.history, columns=["ts", "equity"]).set_index("ts")
            df["returns"] = df["equity"].pct_change().fillna(0.0)
            df["cum_return"] = (1 + df["returns"]).cumprod() - 1
            return df
    def sharpe_ratio(returns, freq=252):
        if len(returns) < 2:
            return 0.0
        mu = returns.mean() * freq
        sigma = returns.std() * math.sqrt(freq)
        return mu / sigma if sigma != 0 else 0.0
    def max_drawdown(equity_curve):
        roll_max = equity_curve.cummax()
        drawdown = (equity_curve - roll_max) / roll_max
        return drawdown.min()

print('Ready')


## Equations & Strategy

Given two price series P1_t and P2_t, build the spread S_t = P1_t - beta * P2_t (beta estimated by regression). Compute z-score: z_t = (S_t - mu)/sigma (rolling). Entry when |z| > entry_z, exit when |z| < exit_z.
Positions: short spread when z > entry_z (short P1, long P2), long spread when z < -entry_z (long P1, short P2).

In [ ]:
# Synthetic cointegrated pair
np.random.seed(42)
T = 500
x = np.cumsum(np.random.normal(0, 0.5, T)) + 50
noise = np.random.normal(0, 0.2, T)
beta_true = 1.8
y = beta_true * x + noise

dfp = pd.DataFrame({'P1': x, 'P2': y})
dfp.head()


In [ ]:
# Estimate beta via OLS on a rolling window
window = 50
betas = []
spread = []
for i in range(window, T):
    y_slice = dfp['P1'].iloc[i-window:i]
    x_slice = dfp['P2'].iloc[i-window:i]
    # beta = cov(P2,P1)/var(P2)
    beta = np.cov(x_slice, y_slice, ddof=0)[0,1] / np.var(x_slice)
    betas.append(beta)
    s = dfp['P1'].iloc[i] - beta * dfp['P2'].iloc[i]
    spread.append(s)

betas = np.array(betas)
spread = np.array(spread)
mu = pd.Series(spread).rolling(20).mean()
sigma = pd.Series(spread).rolling(20).std()
z = (pd.Series(spread) - mu) / sigma

z = z.fillna(0)


In [ ]:
# Backtest on the generated signals
entry_z = 2.0
exit_z = 0.5
bt = Backtest(100000)
ts = 0
position = 0  # +1 long spread, -1 short spread
size = 10.0
equity_prices = []
for i in range(len(z)):
    zi = z[i]
    idx = i + window  # map back to original timestamp
    p1 = dfp['P1'].iloc[idx]
    p2 = dfp['P2'].iloc[idx]
    if position == 0:
        if zi > entry_z:
            # short spread: sell P1, buy P2
            bt.execute_trade(ts, 'P1', size, p1, 'sell')
            bt.execute_trade(ts, 'P2', size*betas[i], p2, 'buy')
            position = -1
        elif zi < -entry_z:
            bt.execute_trade(ts, 'P1', size, p1, 'buy')
            bt.execute_trade(ts, 'P2', size*betas[i], p2, 'sell')
            position = 1
    elif position == 1:
        if abs(zi) < exit_z:
            # close
            bt.execute_trade(ts, 'P1', size, p1, 'sell')
            bt.execute_trade(ts, 'P2', size*betas[i], p2, 'buy')
            position = 0
    elif position == -1:
        if abs(zi) < exit_z:
            bt.execute_trade(ts, 'P1', size, p1, 'buy')
            bt.execute_trade(ts, 'P2', size*betas[i], p2, 'sell')
            position = 0
    # mark-to-market using P1 mid and P2 mid
    bt.mark_to_market(ts, {'P1': p1, 'P2': p2})
    ts += 1

df = bt.results_df()
print('Trades:', len(bt.trades))
print('Final equity:', df['equity'].iloc[-1])
print('Sharpe:', sharpe_ratio(df['returns']))
print('Max Drawdown:', max_drawdown(df['equity']))


In [ ]:
# Visualise spread z-score and equity
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(z)), y=z, mode='lines', name='z-score'))
fig.add_hline(y=entry_z, line_dash='dash', line_color='red')
fig.add_hline(y=-entry_z, line_dash='dash', line_color='red')
fig.add_hline(y=0, line_dash='dot', line_color='black')
fig.update_layout(title='Spread z-score')
fig.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df.index, y=df['equity'], mode='lines', name='Equity'))
fig2.update_layout(title='Pair Trading Backtest Equity')
fig2.show()


### Comments
- This toy example demonstrates basic pair-trading logic and metrics. For production/real market usage: perform robust cointegration testing, use transaction costs, position sizing by volatility, and live monitoring.
- The notebook is ready to accept real price series from connectors and to use Rust-accelerated analytics if available.